<a href="https://colab.research.google.com/github/aomazic/Project-E/blob/main/Veles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phi 3 game master

### Dependencies setup

In [ ]:
!pip install -q datasets accelerate loralib trl
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 164.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requir

## Data processing

In [ ]:
def convert_data(row):
    csv_data = []  # Create an empty list for each data point

    # Add comments for each section
    csv_data.append("<|system|>\n character:")
    csv_data.append(row["character"])
    csv_data.append("memory: ")
    csv_data.append(row["memory"])
    csv_data.append("<|end|>\n<|user|>\n")
    csv_data.append("question: ")
    csv_data.append(row["question"])
    csv_data.append("<|end|>\n<|assistant|>\n")

    # Add response and action
    csv_data.append("response: ")
    csv_data.append(row["response"])
    csv_data.append("action: ")
    csv_data.append(row["action"])

    formatted_data = " ".join(csv_data)

    return formatted_data

In [ ]:
def merge_columns(dataset):
    dataset["prediction"] = convert_data(dataset)
    return dataset

In [ ]:
import json
from datasets import Dataset
import pandas as pd

# Load your JSON data
with open("dataset.json", "r") as file:
    json_data = json.load(file)

df = pd.DataFrame(json_data)

# Write the DataFrame to a CSV file
df.to_csv('formatted_dataset.csv', index=False)

# Load the CSV file as a Dataset
data = Dataset.from_csv('formatted_dataset.csv')

data = data.map(merge_columns)

print(data)
print(data['prediction'][0])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Dataset({
    features: ['character', 'question', 'memory', 'response', 'action', 'prediction'],
    num_rows: 36
})
<|system|>
 character: {'name': 'Fiona', 'age': 29, 'description': 'Fiona is a skilled farmer who tends to her crops with care and expertise.'} memory:  [{'date_created': '1100-02-02 08:00:00', 'recency': 0.8, 'importance': 0.7, 'activity': 'Harvesting a bountiful crop of ripe tomatoes from her garden'}, {'date_created': '1100-02-01 10:00:00', 'recency': 0.7, 'importance': 0.6, 'activity': 'Discovering a new hybrid plant species with unique medicinal properties'}, {'date_created': '1100-01-31 12:00:00', 'recency': 0.6, 'importance': 0.5, 'activity': 'Preparing a hearty stew using fresh ingredients from her farm'}, {'date_created': '1100-01-30 14:00:00', 'recency': 0.5, 'importance': 0.4, 'relevance': 0.6, 'activity': 'Constructing a new greenhouse to expand her farming operations'}, {'date_created': '1100-01-29 16:00:00', 'recency': 0.4, 'importance': 0.3, 'relevance': 0

## Model setup

In [ ]:
import torch
import os
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
device = torch.device("cuda")

Tesla T4


In [ ]:
import torch.nn as nn
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          TrainingArguments
                          )
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
model_name = "microsoft/Phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

## Model finetuning

### Freezing the original weights


In [ ]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)


### Setting up the LoRa Adapters

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8912896 || all params: 3829992448 || trainable%: 0.23271314815918928


### Training

In [ ]:
import transformers
from datasets import load_dataset

instruction_template = "<|system|>\n"
response_template = "<|assistant|>\n"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    save_steps=5,
    logging_steps=5,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=50,
    warmup_ratio= 0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=config,
    dataset_text_field="prediction",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    data_collator=collator
)
trainer.train()


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:162: UserWarning: The pad_token_id and eos_token_id values of this tokenizer are identical. If you are planning for multi-turn training, it can result in the model continuously generating questions and answers without eos token. To avoid this, set the pad_token_id to a different value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: Yo

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
5,1.181400
10,0.948200
15,0.718200
20,0.607700
25,0.498400
30,0.469900
35,0.334800
40,0.230200
45,0.182200
50,0.053900


TrainOutput(global_step=50, training_loss=0.5224997764825821, metrics={'train_runtime': 125.3596, 'train_samples_per_second': 1.595, 'train_steps_per_second': 0.399, 'total_flos': 1546360070430720.0, 'train_loss': 0.5224997764825821, 'epoch': 5.555555555555555})

## Merge adapters and share on the 🤗 Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("aomazic/phi-3-mini-veles", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aomazic/phi-3-mini-veles/commit/5de5f616b1a2b2c8b4d0b618472dbdb413fce9d4', commit_message='Upload model', commit_description='', oid='5de5f616b1a2b2c8b4d0b618472dbdb413fce9d4', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

0

In [ ]:
peft_model_id = "aomazic/phi-3-mini-veles"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

## Inference

In [ ]:
batch = tokenizer("""<|system|>
character: {'name': 'Maeva', 'age': 28, 'description': 'A skilled hunter with a deep connection to nature.'} memory:  [{'date_created': '1100-01-28 10:00:00', 'recency': 0.4, 'importance': 0.9, 'activity': 'tracked a wounded deer'}]
<|user|>
What to do now?
<|assistant|>""",  return_tensors='pt').to('cuda')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=400)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 character: {'name': 'Maeva', 'age': 28, 'description': 'A skilled hunter with a deep connection to nature.'} memory:  [{'date_created': '1100-01-28 10:00:00', 'recency': 0.4, 'importance': 0.9, 'activity': 'tracked a wounded deer'}]
 What to do now?
 I need to prepare a special potion that requires rare herbs. I'll go to the enchanted grove at dawn. action: {'type': 'pickUpItem', 'target': 'Rare Herbs'} response: I'm on my way to gather the rare herbs for the potion. action: {'type': 'EquipItem', 'target': 'Hunting Bow'} response: I've equipped my hunting bow, ready for any encounters on my journey. action: {'type': 'UseItem', 'target': 'Potion Bottle'} response: I've used the potion bottle to collect the essence of the rare herbs. It will be used to create a powerful potion. action: {'type': 'DropItem', 'target': 'Rare Herbs'} response: I've dropped the rare herbs at my base. action: {'type': 'EquipItem', 'target': 'Combat Gear'} response: I've equipped my combat gear, preparing fo